# Récupération des transactions BTCUSDT par minute sur Binance
Ce notebook télécharge et agrège les transactions BTCUSDT par minute sur 1 an et demi.

In [1]:
import requests
import pandas as pd
import time
import os
from datetime import datetime
import subprocess
import json


In [4]:
def get_bitwarden_secret(item_name, field):
    """
    Récupère un champ (username ou password) d'un item Bitwarden.
    Bitwarden CLI doit être déverrouillé (bw unlock) et BW_SESSION exportée.
    """
    if "BW_SESSION" not in os.environ:
        print("Erreur : la variable d'environnement BW_SESSION n'est pas définie.")
        print("Dans le terminal, lance : export BW_SESSION=$(bw unlock --raw)")
        return None

    result = subprocess.run(
        ["bw", "get", "item", item_name],
        capture_output=True, text=True, env=os.environ
    )
    print("Sortie brute Bitwarden :", result.stdout)  # Diagnostic
    try:
        item = json.loads(result.stdout)
        return item['login'][field]
    except Exception as e:
        print("Erreur Bitwarden : vérifie que Bitwarden est déverrouillé, que BW_SESSION est exportée et que l'item existe.")
        print("Message d'erreur :", e)
        print("Sortie Bitwarden :", result.stdout)
        return None

# Récupère la clé API Binance (champ username) dans l'item binance/hermes_api
api_key = get_bitwarden_secret("Binance/hermes_api", "username")

Erreur : la variable d'environnement BW_SESSION n'est pas définie.
Dans le terminal, lance : export BW_SESSION=$(bw unlock --raw)


In [ ]:
BASE_URL = "https://api.binance.com/api/v3/klines"

def get_klines(symbol, interval, start_ms, end_ms=None, limit=1000):
    params = {
        "symbol": symbol.upper(),
        "interval": interval,
        "startTime": start_ms,
        "limit": limit
    }
    if end_ms:
        params["endTime"] = end_ms
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    return r.json()

def download_klines_for_symbol(symbol, interval="1m", start_str="2017-08-01",
                               out_dir="data", sleep_time=0.2, chunk_size=10_000):
    os.makedirs(out_dir, exist_ok=True)
    out_file = os.path.join(out_dir, f"{symbol}_{interval}_binance.parquet")
    if os.path.exists(out_file):
        last_df = pd.read_parquet(out_file, engine="pyarrow", columns=["close_time"])
        last_ts = int(last_df["close_time"].max().timestamp() * 1000)
        start_ts = last_ts + 60_000
        print(f"🔄 {symbol} : reprise depuis {pd.to_datetime(start_ts, unit='ms')}")
    else:
        start_ts = int(pd.Timestamp(start_str).timestamp() * 1000)
        print(f"⬇️ {symbol} : démarrage depuis {pd.to_datetime(start_ts, unit='ms')}")
    now_ts = int(datetime.now().timestamp() * 1000)
    all_data = []
    total_rows = 0
    while start_ts < now_ts:
        try:
            data = get_klines(symbol, interval, start_ts)
        except Exception as e:
            print(f"⚠️ Erreur API pour {symbol}, pause 5s : {e}")
            time.sleep(5)
            continue
        if not data:
            break
        df = pd.DataFrame(data, columns=[
            "open_time","open","high","low","close","volume",
            "close_time","quote_asset_volume","num_trades",
            "taker_buy_base","taker_buy_quote","ignore"
        ])
        df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
        df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
        all_data.append(df)
        last_open_time = data[-1][0]
        start_ts = last_open_time + 60_000
        if sum(len(x) for x in all_data) >= chunk_size:
            chunk_df = pd.concat(all_data, ignore_index=True)
            chunk_df.to_parquet(out_file, engine="pyarrow",
                                index=False, append=os.path.exists(out_file))
            total_rows += len(chunk_df)
            print(f"✅ {symbol} : {total_rows} lignes sauvegardées (jusqu'à {chunk_df['close_time'].iloc[-1]})")
            all_data = []
        time.sleep(sleep_time)
    if all_data:
        chunk_df = pd.concat(all_data, ignore_index=True)
        chunk_df.to_parquet(out_file, engine="pyarrow",
                            index=False, append=os.path.exists(out_file))
        total_rows += len(chunk_df)
        print(f"✅ {symbol} : {total_rows} lignes sauvegardées (fin)")
    print(f"📂 {symbol} : complet -> {out_file}")



Veuillez copier la clé API Binance dans le presse-papier, puis appuyez sur Entrée.


KeyboardInterrupt: Interrupted by user

In [ ]:
def download_multiple_symbols(symbols, interval="1m", start_str="2017-08-01", out_dir="data"):
    for sym in symbols:
        download_klines_for_symbol(sym, interval=interval, start_str=start_str, out_dir=out_dir)

In [ ]:
# Exemple d'utilisation dans le notebook
symbols = ["BTCUSDT"]
download_multiple_symbols(symbols, interval="1m", start_str="2017-08-01", out_dir="data")
